In [88]:
from __future__ import print_function, absolute_import, division
from keras.initializers import Initializer
from keras import backend as K
from keras.engine.base_layer import Layer
from keras.engine.topology import InputSpec
import tensorflow as tf,keras
from keras.layers.merge import Concatenate
from keras.utils import conv_utils
from keras.layers import Input, MaxPooling1D,MaxPooling2D,Conv1D,Activation,Dense, activations, initializers,Conv2D
from keras.layers import Flatten, regularizers,Reshape, constraints, Dropout,Multiply,Add
from keras.layers.core import Lambda
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, SGD
from keras.models import Model
from keras.utils import plot_model
import numpy as np
from scipy.fftpack import dct
from keras.backend.common import normalize_data_format
from keras.layers.merge import Multiply
from keras.utils import to_categorical
from keras.callbacks import LearningRateScheduler, CSVLogger, ModelCheckpoint, Callback
from Gradient_Reverse_Layer import GradientReversal
from custom_layers import Conv1D_gammatone, Conv1D_zerophase,Conv1D_gammatone_coeff
import scipy
from keras.constraints import max_norm
from keras.regularizers import l2
from keras import initializers
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda
from keras.optimizers import Adam, SGD # Nadam, Adamax
from CustomTensorBoard import TensorBoard
from custom_layers import Conv1D_zerophase_linear, Conv1D_linearphase, Conv1D_zerophase,\
    DCT1D, Conv1D_gammatone, Conv1D_linearphaseType, Attention
from keras.layers import Input, Conv1D, MaxPooling1D, Dense, Dropout, Flatten, Activation, AveragePooling1D
from keras import initializers
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.regularizers import l2
from keras.constraints import max_norm
from keras.optimizers import Adam, SGD # Nadam, Adamax
import numpy as np
import tables,h5py
from Gradient_Reverse_Layer import GradientReversal
from ResultAnalyser import Result
from utils import Confused_Crossentropy


# import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.4
# set_session(tf.Session(config=config))
# from clr_callback import CyclicLR
# import dill
from BalancedDannAudioDataGenerator import BalancedAudioDataGenerator, AudioDataGenerator
import os,time
from scipy.io import loadmat
import numpy as np
np.random.seed(1)
import math
import pandas as pd
import tables,h5py
from datetime import datetime
import argparse
from keras.utils import plot_model
# from Heartnet import heartnet,getAttentionModel
from collections import Counter
# from torchviz import make_dot
def to_numpy(x):
    return x.cpu().detach().numpy()
def plotf(x):
    plt.plot(to_numpy(x))
from utils import log_macc, results_log
from dataLoader import reshape_folds
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical
import matplotlib.pyplot as plt
# import seaborn as sns
# import Evaluator
import dataLoader
# from custom_layers import Attention
from sklearn.model_selection import train_test_split

In [2]:
class MFCC(Layer): 
    def __init__(self, rank,
                 filters,
                 kernel_size,
                 output_format='signal',
                 strides=1,
                 padding='valid',
                 data_format=None,
                 dilation_rate=1,
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        super(MFCC, self).__init__(**kwargs)
        self.rank = rank
        self.filters = filters
        self.kernel_size = conv_utils.normalize_tuple(kernel_size, rank,
                                                      'kernel_size')
        self.strides = conv_utils.normalize_tuple(strides, rank, 'strides')
        self.padding = conv_utils.normalize_padding(padding)
        self.data_format = K.normalize_data_format(data_format)
        self.kernel_initializer = kernel_initializer
        self.output_format = output_format
        self.input_spec = InputSpec(ndim=self.rank + 2)

    def build(self, input_shape):
        if self.data_format == 'channels_first':
            channel_axis = 1
        else:
            channel_axis = -1
        if input_shape[channel_axis] is None:
            raise ValueError('The channel dimension of the inputs '
                             'should be defined. Found `None`.')
        input_dim = input_shape[channel_axis]
        kernel_shape = self.kernel_size + (input_dim, self.filters)
    
        
        self.kernel = self.add_weight(shape=kernel_shape,
                                      initializer=self.kernel_initializer,
                                      name='kernel',trainable=False)
        # Set input spec.
        self.input_spec = InputSpec(ndim=self.rank + 2,
                                    axes={channel_axis: input_dim})
        self.built = True

    def call(self, inputs):
        outputs = K.pow(K.abs(inputs),2)
        outputs = K.conv1d(
            outputs,
            self.kernel,
            strides=self.strides[0],
            padding=self.padding,
            data_format=self.data_format)
        outputs = K.log(outputs)
#         outputs = tf.signal.dct(outputs,type=2,axis=-1,norm='ortho')
        if(self.output_format=='image'):
            outputs = K.expand_dims(outputs,axis=-1)
        return outputs

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_last':
            space = input_shape[1:-1]
            new_space = []
            for i in range(len(space)):
                new_dim = conv_utils.conv_output_length(
                    space[i],
                    self.kernel_size[i],
                    padding=self.padding,
                    stride=self.strides[i])
                new_space.append(new_dim)
            if(self.output_format=='image'):
                return (input_shape[0],) + tuple(new_space) + (self.filters,) + (1,)
            else:
                return (input_shape[0],) + tuple(new_space) + (self.filters,)
            
        if self.data_format == 'channels_first':
            raise NotImplementedError("Output formate image/signal not handled")
            space = input_shape[2:]
            new_space = []
            for i in range(len(space)):
                new_dim = conv_utils.conv_output_length(
                    space[i],
                    self.kernel_size[i],
                    padding=self.padding,
                    stride=self.strides[i])
                new_space.append(new_dim)
            return (input_shape[0], self.filters) + tuple(new_space)
    
    def get_config(self):
        config = {
            'rank': self.rank,
            'filters': self.filters,
            'kernel_size': self.kernel_size,
            'strides': self.strides,
            'padding': self.padding,
            'data_format': self.data_format,
            'output_format':self.output_format,
            'kernel_initializer' : initializers.serialize(self.kernel_initializer)
        }
        base_config = super(MFCC, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


In [3]:
def to_colormap(x):
    import matplotlib._cm_listed as cmlist
    cm = tf.convert_to_tensor(np.array(cmlist._viridis_data))
    b = 255*(x-K.min(x))/(K.max(x)-K.min(x))
    b = K.cast(b,dtype='int64')
    wow = K.gather(cm,b)
    return K.cast(wow,dtype='float32')
def colormap_output_shape(input_shape):
    return tuple( list(input_shape)+[3] )

class TO_colormap(Layer):

    def __init__(self,**kwargs):
        
        super(TO_colormap, self).__init__(**kwargs)
        import matplotlib._cm_listed as cmlist
        self.cm = tf.convert_to_tensor(np.array(cmlist._viridis_data))
        self.input_spec = InputSpec(min_ndim=3)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernelA = self.add_weight(name='kernelA',
                                       shape=(input_shape[1], 3),
                                       initializer='uniform')
        
        self.built = True
        super(TO_colormap, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        b = 255*(x-K.min(x))/(K.max(x)-K.min(x))
        b = K.cast(b,dtype='int64')
        wow = K.gather(self.cm,b)
        return K.cast(wow,dtype='float32')
        
    def compute_output_shape(self, input_shape):
        return tuple( list(input_shape)+[3] )

In [4]:
class mfcc_kernel_init(Initializer):
#     def __init__(self):
        
    def __call__(self, shape, dtype=K.floatx()):
        self.shape = shape
        (kernel_size,in_channels,out_channels) = shape
        if(in_channels!=out_channels):
            raise ValueError("Input and Output Channels must be same. Got {0} input channels and {0} output channels".format(in_channels,out_channels))
        mat = K.eye(in_channels,dtype=dtype)
        mat_n = [mat for i in range(kernel_size)]
        return K.stack(mat_n)
    def get_config(self):
        return {
            "shape":self.shape
        }
# def mfcc_kernel_init(shape, dtype=K.floatx()):
#     (kernel_size,in_channels,out_channels) = shape
#     if(in_channels!=out_channels):
#         raise ValueError("Input and Output Channels must be same. Got {0} input channels and {0} output channels".format(in_channels,out_channels))
#     mat = K.eye(in_channels,dtype=dtype)
#     mat_n = [mat for i in range(kernel_size)]
#     return K.stack(mat_n)
class Freq_Init(Initializer):
    def __init__(self, minf=0., maxf=500):
        self.minf = minf
        self.maxf = maxf
    def __call__(self, shape, dtype=K.floatx()):
        (kernel_size,in_channels) = shape
        start = self.hz2mel(self.minf)
        end = self.hz2mel(self.maxf)
        n = ( end-start)/(kernel_size-1)
        mel =  K.expand_dims(K.arange(start,end+1,n,dtype=dtype),axis=1)
        return self.mel2hz(mel)
    def hz2mel(self,hz):
        return 2595 * (K.log(1.0+(hz*1.0)/700.)/K.log(10.0))
    def mel2hz(self,mel):
        return 700*(10**(mel/2595.0)-1)
    def get_config(self):
        return {
            'minf': self.minf,
            'maxf': self.maxf
        }
def hz2mel(hz):
    return 2595 * np.log10(1+hz/700.)
def mel2hz(mel):
    return 700*(10**(mel/2595.0)-1)
def erb(f):
    return 24.7*(4.37*10**-3*f+1)
class beta_init(Initializer):
    def __init__(self, val = 100):
        self.val = val
    def __call__(self,shape,dtype=K.floatx()):
        (kernel_size,in_channels) = shape
        beta_weights = tf.convert_to_tensor(np.ones((kernel_size,1))*self.val,dtype=K.floatx())
        return beta_weights
    def get_config(self):
        return {
            'val': self.val
        }
# def beta_init(shape, dtype=K.floatx()):
#     (kernel_size,in_channels) = shape
#     beta_weights = tf.convert_to_tensor(np.ones((kernel_size,1))*100,dtype=K.floatx())
#     return beta_weights

In [5]:
def branch(input_tensor,num_filt,filters,kernel_size,random_seed,padding,bias,maxnorm,l2_reg,
           eps,bn_momentum,activation_function,dropout_rate,subsam,trainable):

    num_filt1, num_filt2 = num_filt
    t = Conv1D(num_filt1*filters, kernel_size=kernel_size,
                kernel_initializer=initializers.he_normal(seed=random_seed),
                padding=padding,
                use_bias=bias,
                kernel_constraint=max_norm(maxnorm),
                trainable=trainable,
                kernel_regularizer=l2(l2_reg))(input_tensor)
    t = BatchNormalization(epsilon=eps, momentum=bn_momentum, axis=-1)(t)
    t = Activation(activation_function)(t)
    t = Dropout(rate=dropout_rate, seed=random_seed)(t)
    t = MaxPooling1D(pool_size=subsam)(t)
    t = Conv1D(num_filt2*filters, kernel_size=kernel_size,
               kernel_initializer=initializers.he_normal(seed=random_seed),
               padding=padding,
               use_bias=bias,
               trainable=trainable,
               kernel_constraint=max_norm(maxnorm),
               kernel_regularizer=l2(l2_reg))(t)
    t = BatchNormalization(epsilon=eps, momentum=bn_momentum, axis=-1)(t)
    t = Activation(activation_function)(t)
    t = Dropout(rate=dropout_rate, seed=random_seed)(t)
    t = MaxPooling1D(pool_size=subsam)(t)
    # t = Flatten()(t)
    return t

In [6]:
def Network(kernel_size=5,fs=1000,winlen=0.025,winstep=0.01,filters=64,random_seed=1,padding='valid',bias=False,
           lr=0.0012843784,lr_decay=0.0001132885,subsam=2,num_filt=(8,4),num_dense=20,trainable=True,batch_size=1024,
           l2_reg=0.0,l2_reg_dense=0.0,bn_momentum=0.99,dropout_rate=0.5,dropout_dense=0.0,eps = 1.1e-5,maxnorm=10000,
           activation_function='relu'):
    input = Input(shape=(2500, 1))
    t = Conv1D_gammatone_coeff(kernel_size=81, strides=1,filters=filters,
                         fsHz=fs,use_bias=False,padding='same',
                         fc_initializer=Freq_Init(minf=0.0,maxf=fs/2),
                         amp_initializer=initializers.constant(10**4),
                        beta_initializer=beta_init(val=100),name="gamma"
                        )(input)
    
#     t = Conv1D_gammatone(kernel_size=81, strides=1,filters=filters,
#                          fsHz=fs,use_bias=False,padding='same',
#                          fc_initializer=Freq_Init(minf=50.0,maxf=fs/2),
#                          amp_initializer=initializers.constant(10**4),
#                         beta_initializer=beta_init(val=100),name="gamma"
#                         )(input)
    t = branch(t,num_filt,filters,kernel_size,random_seed,padding,bias,maxnorm,l2_reg,
           eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)
    
    t = BatchNormalization(epsilon=eps, momentum=bn_momentum, axis=-1)(t)
    
#     t = branch(t,num_filt,kernel_size,random_seed,padding,bias,maxnorm,l2_reg,
#            eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)

#     t = res_block(t,32,kernel_size,1,'same',random_seed,bias,maxnorm,l2_reg,
#            eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)

#     t = res_block(t,64,kernel_size,2,'same',random_seed,bias,maxnorm,l2_reg,
#            eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)
    t = Flatten()(t)
    t = Dense(20,
                   activation=activation_function,
                   kernel_initializer=initializers.he_normal(seed=random_seed),
                   use_bias=bias,
                   kernel_constraint=max_norm(maxnorm),
                   kernel_regularizer=l2(l2_reg_dense),
                   name = 'class_dense')(t)
    t = Dense(2, activation='softmax', name="class")(t)
    opt = SGD(lr=.001,decay=.001)
    
    model = Model(inputs=input, outputs=t)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [7]:
kernel_size=5
fs=1000
winlen=0.025
winstep=0.01
filters=64

In [8]:
model = Network(kernel_size,fs,winlen,winstep,filters)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
<tf.Variable 'gamma/weight:0' shape=(81, 1, 64) dtype=float32_ref>
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2500, 1)           0         
_________________________________________________________________
gamma (Conv1D_gammatone_coef (None, 2500, 64)          5184      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2496, 512)         163840    
_________________________________________________________________
batch_normalization_1 (Batch (None, 2496, 512)         2048      
_________________________________________________________________
activation_1 (Activation)    (None, 2496, 512)         0         
____________________

In [100]:
import h5py
path = '../data/fold0_noFIR.mat'
data = h5py.File(path, 'r')
x_train = data['trainX'][:].astype('float32')
x_train = np.expand_dims(x_train.transpose(),2)
x_val = data['valX'][:].astype('float32')
x_val = np.expand_dims(x_val.transpose(),2)
y_train = data['trainY'][:].astype('int32')
y_train = y_train.transpose()
y_train = y_train[:,0]
y_train[y_train<0] = 0
y_val = data['valY'][:].astype('int32')
y_val = y_val.transpose()
y_val = y_val[:,0]
y_val[y_val<0] = 0
val_parts = data['val_parts'][:].astype('int32').squeeze(0)

In [101]:
y_train = to_categorical(y_train,2)
y_val = to_categorical(y_val,2)

In [102]:
val_files = data['val_files'][:].astype('int32').squeeze(0)
train_files = data['train_files'][:].astype('int32').squeeze(0)

In [53]:
x_train = x_train[2000:2100]
y_train = y_train[2000:2100]
train_files = train_files[2000:2100]
x_train.shape,y_train.shape,x_val.shape,y_val.shape

((100, 2500, 1), (100, 2), (6710, 2500, 1), (6710, 2))

In [103]:
domains = list(Counter(train_files).keys())
domainClass = [(cls,dfc) for cls in range(2) for dfc in domains]
meta_labels = [domainClass.index((cl,df)) for (cl,df) in zip(np.argmax(y_train,1),train_files)]

In [104]:
Counter(meta_labels)

Counter({6: 9075,
         0: 2883,
         1: 2790,
         7: 446,
         8: 1080,
         2: 223,
         9: 497,
         3: 294,
         4: 55347,
         10: 2470,
         5: 3294,
         11: 1411})

In [106]:
batch_size = 40
random_seed = 2
datagen = BalancedAudioDataGenerator(shift=.1)
flow = datagen.flow(x_train, y_train,
                meta_label=meta_labels,
                batch_size=batch_size, shuffle=True,
                seed=random_seed)

[ 2883  2790   223   294 55347  3294  9075   446  1080   497  2470  1411]
[ 0  1  2  3  4  5  6  7  8  9 10 11]
Chunk size selected as 3


In [107]:
log_name = "heartnet64 "+str(datetime.now()).replace(':','.')
log_dir = "../../Heartnet_Results/logs/gammatone_torch_layer/"
model_dir = log_dir+log_name+"/weights/"
csv_logger = CSVLogger(log_dir + log_name + '/training.csv')
tensbd = TensorBoard(log_dir=log_dir + log_name,
                     batch_size=batch_size, histogram_freq = 3,
                     write_grads=True,
                     # embeddings_freq=99,
                     # embeddings_layer_names=embedding_layer_names,
                     # embeddings_data=x_val,
                     # embeddings_metadata=metadata_file,
                     write_images=False)


In [108]:
if not os.path.exists(log_dir + log_name):
    os.makedirs(log_dir + log_name)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

In [109]:
# model_json = model.to_json()
# with open(model_dir + log_name+"/model.json", "w") as json_file:
#     json_file.write(model_json)

In [110]:
model_dir

'../../Heartnet_Results/logs/gammatone_torch_layer/heartnet64 2020-05-13 03.08.51.172668/weights/'

In [111]:
checkpoint_name = model_dir + 'weights.{epoch:04d}-{val_acc:.4f}.pickle'
outlayer = ''
val_outlayer_acc = 'val_'+outlayer+'acc'
modelcheckpnt = ModelCheckpoint(filepath=checkpoint_name,
                monitor=val_outlayer_acc,save_best_only=False, mode='max') 

In [112]:
import pickle
class SaveWeights(Callback):
    def __init__(self, checkpoint_name, verbose=0):
        super(SaveWeights, self).__init__()
        self.checkpoint_name = checkpoint_name
    def on_epoch_end(self, epoch,logs):
        d = {x.name:x.get_weights() for x in self.model.layers}
        with open(self.checkpoint_name.format(epoch=epoch+1, val_acc=logs['val_acc']) , 'wb') as f:
            pickle.dump(d,f)
saveweight = SaveWeights(checkpoint_name)

In [113]:
epochs = 200
verbose = 1
model.fit_generator(flow,
#                 steps_per_epoch=len(x_train) // batch_size,
                steps_per_epoch=flow.steps_per_epoch,
                # max_queue_size=20,
                use_multiprocessing=False,
                epochs=epochs,
                verbose=verbose,
                shuffle=True,
                callbacks=[log_macc(val_parts, decision='majority',verbose=verbose,val_files=None,wav_files=None,checkpoint_name = checkpoint_name),
                           tensbd, csv_logger],
                validation_data=(x_val,y_val),
                initial_epoch=0,
                )

Check point  ../../Heartnet_Results/logs/gammatone_torch_layer/heartnet64 2020-05-13 03.08.51.172668/weights/weights.{epoch:04d}-{val_acc:.4f}.pickle
Epoch 1/200
6710/6710 [==============================] - 110s 16ms/step
TN:21,FP:125,FN:6,TP:132,Macc:0.5501786342436907,F1:0.6683493923783347
Epoch 2/200
6710/6710 [==============================] - 110s 16ms/step
TN:21,FP:125,FN:7,TP:131,Macc:0.5465554461266985,F1:0.6649695755227105
Epoch 3/200
6710/6710 [==============================] - 110s 16ms/step
TN:21,FP:125,FN:7,TP:131,Macc:0.5465554461266985,F1:0.6649695755227105
Epoch 4/200
6710/6710 [==============================] - 110s 16ms/step
TN:21,FP:125,FN:7,TP:131,Macc:0.5465554461266985,F1:0.6649695755227105
Epoch 5/200
6710/6710 [==============================] - 110s 16ms/step
TN:21,FP:125,FN:7,TP:131,Macc:0.5465554461266985,F1:0.6649695755227105
Epoch 6/200
6710/6710 [==============================] - 110s 16ms/step
TN:22,FP:124,FN:7,TP:131,Macc:0.5499801034029229,F1:0.666661616

KeyboardInterrupt: 

In [65]:
import utils,importlib
utils = importlib.reload(utils)
from utils import log_macc

In [85]:
for x in model.layers:
    print(x.name,len(x.get_weights()))

input_1 0
gamma 1
conv1d_1 1
batch_normalization_1 4
activation_1 0
dropout_1 0
max_pooling1d_1 0
conv1d_2 1
batch_normalization_2 4
activation_2 0
dropout_2 0
max_pooling1d_2 0
batch_normalization_3 4
flatten_1 0
class_dense 1
class 2


In [82]:
model.get_layer("gamma").get_weights()

[array([[[-0.365681  ,  0.47806203, -0.01162256, ..., -0.21437405,
          -0.34076563, -0.18346953]],
 
        [[-0.25358862,  0.2660421 , -0.05660003, ..., -0.2784855 ,
          -0.14513469, -0.30135432]],
 
        [[-0.17417923,  0.03223413,  0.07543215, ..., -0.3758643 ,
          -0.02395119, -0.2635922 ]],
 
        ...,
 
        [[ 0.03619101, -0.0510463 , -0.22567493, ..., -0.67171764,
           0.58033925,  0.14520212]],
 
        [[ 0.17033169, -0.02735857, -0.2820525 , ..., -0.65479845,
           0.51989114,  0.19547534]],
 
        [[ 0.2642771 , -0.06712583, -0.22479616, ..., -0.53827477,
           0.48296753,  0.25344288]]], dtype=float32)]